In [ ]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [ ]:
!pip install kaggle deepface numpy pandas opencv-python tensorflow scikit-learn -q

In [ ]:
!kaggle datasets download -d msambare/fer2013

Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013
License(s): DbCL-1.0
 93% 56.0M/60.3M [00:00<00:00, 164MB/s]
100% 60.3M/60.3M [00:00<00:00, 172MB/s]


In [ ]:
!mkdir -p video_data
!unzip fer2013.zip -d video_data > /dev/null 2>&1

In [ ]:
import os
import pandas as pd

base_dir = 'video_data'

def create_df(base_dir, subfolder):
  data = []
  subfolder = os.path.join(base_dir, subfolder)
  for class_folder in os.listdir(subfolder):
      class_path = os.path.join(subfolder, class_folder)
      if os.path.isdir(class_path):
          for image in os.listdir(class_path):
              if image.endswith('.jpg'):
                  image_path = os.path.join(class_path, image)
                  data.append([class_folder, image_path])
  df = pd.DataFrame(data, columns=['class', 'filepath'])
  return df

train_df = create_df(base_dir, 'train')
test_df = create_df(base_dir, 'test')

In [ ]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)

train_df = encode_and_bind(train_df, 'class')
test_df = encode_and_bind(test_df, 'class')

Video Preprocessing  
It is not needed for this FER-2013 dataset, since it is already in required format.

In [ ]:
import cv2
from sklearn.model_selection import train_test_split

def load_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img = img.astype('float32') / 255.0
        return img
    return None

train_df['image'] = train_df['filepath'].apply(load_image)
test_df['image'] = test_df['filepath'].apply(load_image)

test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class_cols = [i for i in train_df.columns if i.startswith('class')]

class FERDataset(Dataset):
    def __init__(self, df):
        self.images = np.stack(df['image'].values)
        self.images = self.images.reshape(-1, 48, 48)  # Shape: (48, 48), no extra channel dimension
        self.labels = df[class_cols].values.argmax(axis=1).astype('int64')

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Convert to a PyTorch tensor and add the channel dimension (1, 48, 48)
        image = torch.FloatTensor(self.images[idx]).unsqueeze(0)  # Adds channel dimension
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return image, label


train_dataset = FERDataset(train_df)
test_dataset = FERDataset(test_df)
val_dataset = FERDataset(val_df)

batch_size = 16

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Training

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

from models import ImageNet

class EmotionTrainer:
    def __init__(
        self,
        model,
        train_loader,
        val_loader,
        optimizer,
        criterion=nn.CrossEntropyLoss(),
        device='cuda' if torch.cuda.is_available() else 'cpu'
    ):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device
        self.history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    def train_epoch(self):
        self.model.train()
        total_loss = 0
        all_preds = []
        all_labels = []

        # Progress bar for training
        pbar = tqdm(self.train_loader, desc='Training')

        for batch_idx, (images, labels) in enumerate(pbar):
            images, labels = images.to(self.device), labels.to(self.device)

            # Zero the gradients
            self.optimizer.zero_grad()

            # Forward pass
            outputs = self.model(images)
            loss = self.criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            self.optimizer.step()

            # Track metrics
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Update progress bar
            pbar.set_postfix({'loss': loss.item()})

        # Calculate epoch metrics
        epoch_loss = total_loss / len(self.train_loader)
        epoch_acc = accuracy_score(all_labels, all_preds)

        return epoch_loss, epoch_acc

    @torch.no_grad()
    def validate(self):
        self.model.eval()
        total_loss = 0
        all_preds = []
        all_labels = []

        for images, labels in tqdm(self.val_loader, desc='Validation'):
            images, labels = images.to(self.device), labels.to(self.device)

            # Forward pass
            outputs = self.model(images)
            loss = self.criterion(outputs, labels)

            # Track metrics
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        # Calculate validation metrics
        val_loss = total_loss / len(self.val_loader)
        val_acc = accuracy_score(all_labels, all_preds)

        # Print detailed classification report
        print("\nClassification Report:")
        print(classification_report(all_labels, all_preds))

        return val_loss, val_acc

    def train(self, num_epochs=133, save_best=True):
        best_val_loss = float('inf')

        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch+1}/{num_epochs}")

            # Train and validate
            train_loss, train_acc = self.train_epoch()
            val_loss, val_acc = self.validate()

            # Update history
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_acc'].append(val_acc)

            # Print epoch summary
            print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
            print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

            # Save best model
            if save_best and val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(self.model.state_dict(), 'best_model.pth')
                print("Saved best model!")

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

model = ImageNet()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


trainer = EmotionTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer = optimizer,
)

trainer.train(num_epochs=113)


Epoch 1/13


Validation: 100%|██████████| 113/113 [00:22<00:00,  5.13it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", 


Classification Report:
              precision    recall  f1-score   support

           0       0.41      0.04      0.08       494
           1       0.00      0.00      0.00        60
           2       0.25      0.05      0.09       525
           3       0.45      0.72      0.55       869
           4       0.35      0.24      0.29       610
           5       0.26      0.47      0.34       623
           6       0.47      0.58      0.52       408

    accuracy                           0.38      3589
   macro avg       0.31      0.30      0.27      3589
weighted avg       0.36      0.38      0.32      3589

Train Loss: 1.7433 | Train Acc: 0.2980
Val Loss: 1.5874 | Val Acc: 0.3767
Saved best model!

Epoch 2/13


Validation: 100%|██████████| 113/113 [00:22<00:00,  5.12it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", 


Classification Report:
              precision    recall  f1-score   support

           0       0.40      0.12      0.19       494
           1       0.00      0.00      0.00        60
           2       0.31      0.05      0.08       525
           3       0.54      0.84      0.66       869
           4       0.34      0.63      0.44       610
           5       0.34      0.26      0.30       623
           6       0.59      0.61      0.60       408

    accuracy                           0.45      3589
   macro avg       0.36      0.36      0.33      3589
weighted avg       0.42      0.45      0.39      3589

Train Loss: 1.5217 | Train Acc: 0.4101
Val Loss: 1.3984 | Val Acc: 0.4497
Saved best model!

Epoch 3/13


...

Validation: 100%|██████████| 113/113 [00:22<00:00,  5.11it/s]



Classification Report:
              precision    recall  f1-score   support

           0       0.45      0.36      0.40       494
           1       0.53      0.15      0.23        60
           2       0.42      0.16      0.23       525
           3       0.74      0.75      0.75       869
           4       0.44      0.58      0.50       610
           5       0.37      0.52      0.43       623
           6       0.67      0.67      0.67       408

    accuracy                           0.52      3589
   macro avg       0.52      0.46      0.46      3589
weighted avg       0.53      0.52      0.51      3589

Train Loss: 1.3128 | Train Acc: 0.4960
Val Loss: 1.2347 | Val Acc: 0.5241
Saved best model!

Epoch 6/13


Validation: 100%|██████████| 113/113 [00:22<00:00,  5.11it/s]



Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.33      0.41       494
           1       0.50      0.08      0.14        60
           2       0.38      0.22      0.28       525
           3       0.74      0.77      0.76       869
           4       0.48      0.54      0.51       610
           5       0.37      0.54      0.44       623
           6       0.63      0.73      0.68       408

    accuracy                           0.53      3589
   macro avg       0.52      0.46      0.46      3589
weighted avg       0.53      0.53      0.52      3589

Train Loss: 1.2808 | Train Acc: 0.5131
Val Loss: 1.1970 | Val Acc: 0.5341
Saved best model!

Epoch 7/13


Training:  72%|███████▏  | 649/898 [06:12<02:57,  1.41it/s, loss=1.36]